# Livros de Python na Amazom em Portugues

Objetivo: verificar o conjunto de livros em portugues da linguagem python, bem como relacionar Total de avaliações e quantidades de estrelas.
Formar base de dados de livros de python de uma plataforma de vendas on-line.

### Bibliotecas:

In [6]:
from selenium.webdriver.common.by import By
import pandas as pd
from rich.console import Console
from selenium.webdriver import ActionChains
from driver_scraping import Scraping
import time
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm
from parsel import Selector

#### Dados e fontes:

* Amazon: https://www.amazon.com.br/
* Richt prints bonitos : https://rich.readthedocs.io/en/stable/introduction.html
* bs4: https://beautiful-soup-4.readthedocs.io/en/latest/
* pandas: https://pandas.pydata.org/docs/
* selenium: https://www.selenium.dev/documentation/overview/
* parsel: https://parsel.readthedocs.io/en/latest/index.html


In [7]:
url = 'https://www.amazon.com.br/'

In [36]:
# funções para bs4 uteis, foi utilizada no começo e depois substituida por parsel.

def pega_page(html_list):
    """Retorna um objeto BS4 a partir 
    de uma geradora de htmls"""
    page = BeautifulSoup(next(html_list), 'html.parser')
     
    return page



def filtro_tag(pagina, tag):
    """Faz filtro de Tag html de um arquivo a fim de 
    diminuir nomes de classes css em objetos que não queremos alcançar"""
    busca = pagina.find_all(tag)
    rtag = BeautifulSoup(str(busca), 'html.parser')
    
    return rtag

# função teste caso a internet caia
def tentar_conect():
    try: 
        host = 'www.google.com'
        s = create_connection((host, 80), 2)
        s.close()
        return "passou"
    except:
        return "não"

# função para transformar lista em dicionário
def descricao(valor):
    """Apos pegar um conjunto de dados 
    possives de rotular ele retorna um dicionário - Para carrocel de dados"""
    aux = pagina.xpath('//ol[@class="a-carousel"]//li[@class="a-carousel-card rpi-carousel-attribute-card"]//span/text()').getall()
    descricao = {}
    for index, i in enumerate(aux):
        if index % 2 == 0:
            descricao[i] = aux[index+1]
    if valor in descricao.keys():
        return descricao[valor]
    elif "dict" in valor:
        return descricao
    else:
        return "vazio"

# Scraping

##### Visita uma url, faz filtro por pesquisa de palavra chave, percorre um index de páginas do resultado pegando os links de produtos. 16 produtos por paginas e por fim gera uma lista de links onde cada item da lista é uma lista de 16 links.

In [9]:
with Scraping(url) as driver:
    print(driver.title)
    
    #pesquisa
    search = driver.find_element(By.NAME, 'field-keywords')
    search.send_keys('Python')
    search.submit()
    
    #checkbox
    icon_box = driver.find_element(By.XPATH,
        '//*[@id="p_n_feature_nine_browse-bin/9754444011"]/span/a/div/label/i')

    icon_box.click()
    time.sleep(5)
    
    # informações de pagina. Numero de Página
    final = driver.find_elements(By.CLASS_NAME, 's-pagination-disabled')
    final  = int(final[1].text)
    cont = 0
    
    lista_links = []
    
    while not cont >= final:
        #elemento para rolar pagina
        index_pages = driver.find_element(By.CLASS_NAME, 's-pagination-strip')
        next_page = driver.find_elements(By.CLASS_NAME,'s-pagination-item')
        
        #rolar até o xpath index pages
        ActionChains(driver).scroll_to_element(index_pages).perform()
        
        # pagina atual
        pagina = driver.page_source
        data = BeautifulSoup(pagina, 'html.parser')
        #filtro por h2 -> tangs a class 
        tag = filtro_tag(data, 'h2')
        links = tag.find_all('a', class_='s-link-style')
        #salvar links pagina atual na lista de links
        lista_links.append(links)

        
        # próximo
        next_page[-1].click()
        cont+=1    

Amazon.com.br | Tudo pra você, de A a Z.


In [37]:
# quantidades de páginas
len(lista_links)

48

In [38]:
# pagina por pagina 16
gerador_links = (i for i in lista_links)

# Parte 2: 
##### Visita cada link de produto e salva a pagina html em um diretório ou raspa os dados diretamente e salva em lista.

In [39]:
autores = []
titulos = []
avaliacoes = []
total_avaliacoes = []
paginas = []
ano = []
editora = []
tipo = []

In [40]:
with Scraping(url) as driver:
    print(driver.title)
    
    # todas as paginas individuais de livros
    paginas = []
    chave = True
    cont = 0
    #index = 0
    while chave:
        
        for link in tqdm(list(next(gerador_links))):
            
            try:
                driver.get(f"https://www.amazon.com.br/{link['href']}")
            except:
                res = "não"
                ct = 0
                while res == "não":
                    time.sleep(20)
                    res = tentar_conect()
                    print(f"Tentando conexão {ct}", end="\r")
                    
                    if res == "passou":
                        driver.get(f"https://www.amazon.com.br/{link['href']}")
                    ct+=1

            # parse do doados
            time.sleep(5)
            
            html = driver.page_source
            
            pagina = Selector(text=html)
            
            autores.append(pagina.xpath('//span[@class="author notFaded"]/a/text()').getall())
            
            titulos.append(pagina.xpath('//span[@id="productTitle"]/text()').get().strip())
            
            avaliacoes.append(pagina.xpath('//a[@class="a-popover-trigger a-declarative"]/span[@class="a-size-base a-color-base"]/text()').get())
            
            total_avaliacoes.append(pagina.xpath('//span[@id="acrCustomerReviewText"]/text()').get())
            
            paginas.append(descricao('Número de páginas'))
            
            ano.append(descricao('Data da publicação'))
            
            editora.append(descricao('Editora'))
            
            tipo.append(pagina.xpath('//span[@id="productSubtitle"]/text()').getall())
            
            #paginas.append(driver.page_source)
            #atual = open(f'./Dados Salvos/Paginas/pagina_de_livro{index}.txt', 'w', encoding="utf-8")
            #atual.write(driver.page_source)
            #index+=1

            
        cont+=1
        if cont == 48:
            chave = False

Amazon.com.br | Tudo pra você, de A a Z.


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [01:55<00:00,  8.25s/it]


# Pré Tratamento dos Dados:

In [42]:
len(titulos)

760

In [43]:
df1 = pd.DataFrame(titulos).rename(columns={0:'Título'})
df2 = pd.DataFrame(autores).rename(columns={0:'Autor'})
df3 = pd.DataFrame(avaliacoes).rename(columns={0:'Nota'})
df4 = pd.DataFrame(total_avaliacoes).rename(columns={0:'Total de Avaliações'})
df5 = pd.DataFrame(paginas).rename(columns={0:'Nº de Páginas'})
df6 = pd.DataFrame(ano).rename(columns={0:'Data'})
df7 = pd.DataFrame(editora).rename(columns={0:'Editora'})
df8 = pd.DataFrame(tipo).rename(columns={0:"Tipo"})

In [44]:
data = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=1)
data.rename(columns={1:"Co Autor", 2:'Co-Autor..'}, inplace=True)

In [45]:
data

,Título,Autor,Co Autor,Co-Autor..,Nota,Total de Avaliações,Nº de Páginas,Data,Editora,Tipo
0,Curso Intensivo de Python: uma Introdução Prát...,Eric Matthes,None,None,"4,9",165 avaliações de clientes,624 páginas,24 abril 2023,Novatec Editora,Capa comum – 24 abril 2023
1,Introdução à Programação com Python – 4ª Ediçã...,Nilo Ney Coutinho Menezes,None,None,"5,0",14 avaliações de clientes,552 páginas,11 março 2024,Novatec Editora,Capa comum – 11 março 2024
2,Pense em Python: Pense Como um Cientista da Co...,Allen B. Downey,None,None,"4,8",562 avaliações de clientes,312 páginas,7 junho 2016,Novatec Editora,Capa comum – 7 junho 2016
3,Python Para Análise de Dados: Tratamento de Da...,Wes McKinney,None,None,"4,8",130 avaliações de clientes,624 páginas,16 março 2023,Novatec Editora,Capa comum – 16 março 2023
4,Data Science do Zero - 2º Edição: Noções Funda...,Joel Grus,None,None,"4,8",725 avaliações de clientes,416 páginas,30 março 2021,Editora Alta Books,Capa comum – 30 março 2021
...,...,...,...,...,...,...,...,...,...,...
755,Humoristas do Reino Unido: Humoristas da Ingla...,Fonte: Wikipedia,None,None,None,None,30 páginas,26 junho 2011,"Books LLC, Wiki Series",Capa comum – 26 junho 2011
756,"Linguagens de script: Python, Euphoria, JavaSc...",Fonte: Wikipedia,None,None,"1,0",1 classificação,50 páginas,25 junho 2011,"Books LLC, Wiki Series",Capa comum – 25 junho 2011
757,"Humor: Desciclopédia, Piada, Monty Python, Eng...",Fonte: Wikipedia,None,None,None,None,30 páginas,27 junho 2011,"Books LLC, Wiki Series",Capa comum – 27 junho 2011
758,Gastaria Tudo com Pizza - Edição com Autógrafo,Pedro Duarte,None,None,"4,9",57 avaliações de clientes,224 páginas,2 dezembro 2019,Pipoca e Nanquim,Capa dura – 2 dezembro 2019


In [46]:
data.to_csv("Dados_livros_python_pre_processados.csv")

In [47]:
s = data['Editora'] == 'Novatec Editora	'

In [49]:
data[s]

,Título,Autor,Co Autor,Co-Autor..,Nota,Total de Avaliações,Nº de Páginas,Data,Editora,Tipo
4,Data Science do Zero - 2º Edição: Noções Funda...,Joel Grus,None,None,"4,8",725 avaliações de clientes,416 páginas,30 março 2021,Editora Alta Books,Capa comum – 30 março 2021
6,Python Para Data Science: e Machine Learning D...,Amilcar Netto,Francisco Maciel,None,"4,5",155 avaliações de clientes,384 páginas,8 julho 2021,Editora Alta Books,Capa comum – 8 julho 2021
9,Começando a Programar em Python Para Leigos,John Paul Mueller,None,None,"4,6",218 avaliações de clientes,391 páginas,5 novembro 2020,Editora Alta Books,"Capa comum – Edição padrão, 5 novembro 2020"
21,Python Para Data Science Para Leigos: os Prime...,John Paul Mueller,Luca Massaron,None,"4,4",125 avaliações de clientes,246 páginas,21 janeiro 2021,Editora Alta Books,"Capa comum – Edição padrão, 21 janeiro 2021"
28,Mãos à Obra: Aprendizado de Máquina com Scikit...,Aurélien Géron,None,None,"4,8",500 avaliações de clientes,640 páginas,6 setembro 2021,Editora Alta Books,Capa comum – 6 setembro 2021
39,Data Science Para Negócios,Foster Provost,Tom Fawcett,None,"4,7",1.287 avaliações de clientes,404 páginas,22 março 2016,Editora Alta Books,Capa comum – 22 março 2016
42,Estatística Prática Para Cientistas de Dados: ...,Andrew Bruce,Peter Bruce,None,"4,7",1.139 avaliações de clientes,392 páginas,1 julho 2019,Editora Alta Books,Capa comum – 1 julho 2019
62,Python e Django: Desenvolvimento web Moderno e...,Francisco Marcelo de Barros Maciel,None,None,"4,5",92 avaliações de clientes,448 páginas,17 agosto 2020,Editora Alta Books,Capa comum – 17 agosto 2020
70,Programação Funcional Para Leigos,John Paul Mueller,None,None,"4,2",19 avaliações de clientes,320 páginas,20 dezembro 2019,Editora Alta Books,Capa comum – 20 dezembro 2019
88,Arquitetura Limpa: o Guia do Artesão Para Estr...,Robert C. Martin,None,None,"4,9",4.143 avaliações de clientes,432 páginas,23 abril 2019,Editora Alta Books,"Capa comum – Edição padrão, 23 abril 2019"
